In [1]:
from typing_extensions import TypedDict, Literal
from langgraph.graph import StateGraph, END, START

In [2]:
class Account_State(TypedDict):
    current_bal : int
    amount : int
    action : Literal["Debit",'Credit']
    balance : None

In [3]:
def credit_amount(state:Account_State) -> Account_State:
    return {'balance':state["current_bal"] + state["amount"]}

In [4]:
def debit_amount(state: Account_State) -> Account_State:
    if state['current_bal'] - state['amount'] < 5000:
        raise ValueError('Balance Cannot be less than 50000 ')
    else:
        return {'balance':state['current_bal'] - state['amount']}


In [6]:
def choose_action(state: Account_State) -> str:
    return state['action']

In [7]:
builder = StateGraph(Account_State)

builder.add_node('Credit_node',credit_amount)
builder.add_node('Debit_node',debit_amount)

builder.add_conditional_edges(START,choose_action,{'Credit':'Credit_node','Debit':'Debit_node'})
builder.add_edge(['Credit_node','Debit_node'],END)

graph = builder.compile()

In [9]:
graph.invoke({"current_bal":76000,'action':'Credit','amount':15000})

{'current_bal': 76000, 'amount': 15000, 'action': 'Credit', 'balance': 91000}

In [10]:
graph.invoke({"current_bal":76000,'action':'Debit','amount':15000})

{'current_bal': 76000, 'amount': 15000, 'action': 'Debit', 'balance': 61000}

In [15]:
# graph.invoke({"current_bal":56000,'action':'Debit','amount':55000})   # This will raise ValueError

In [14]:
print(graph.get_graph().draw_ascii())

              +-----------+               
              | __start__ |               
              +-----------+               
             ...          ..              
            .               ..            
          ..                  ..          
+-------------+           +------------+  
| Credit_node |           | Debit_node |  
+-------------+           +------------+  
             ***          **              
                *       **                
                 **   **                  
               +---------+                
               | __end__ |                
               +---------+                
